# Random Object Placement in Room

In [1]:
from tdw.controller import Controller
from tdw.tdw_utils import TDWUtils
from tdw.add_ons.third_person_camera import ThirdPersonCamera
from tdw.add_ons.image_capture import ImageCapture
from tdw.librarian import ModelLibrarian
from tdw.output_data import Bounds, OutputData

from pathlib import Path
import time
import numpy as np
import math

In [2]:
librarian = ModelLibrarian("models_core.json")
chair_records = []
for record in librarian.records:
    if "chair" in record.name.lower():
        chair_records.append(record.name)
print(chair_records)

['blue_club_chair', 'blue_side_chair', 'brown_leather_dining_chair', 'brown_leather_side_chair', 'chair_annabelle', 'chair_billiani_doll', 'chair_eames_plastic_armchair', 'chair_thonet_marshall', 'chair_willisau_riale', 'dark_red_club_chair', 'emeco_navy_chair', 'green_side_chair', 'lapalma_stil_chair', 'ligne_roset_armchair', 'linbrazil_diz_armchair', 'linen_dining_chair', 'naughtone_pinch_stool_chair', 'red_side_chair', 'tan_lounger_chair', 'tan_side_chair', 'vitra_meda_chair', 'white_club_chair', 'white_lounger_chair', 'wood_chair', 'yellow_side_chair']


In [3]:
import functools

def without_capture(method):
    @functools.wraps(method)
    def wrapper(self, *args, **kwargs):
        # remember if the capture add-on was active
        was_active = self.cap in self.controller.add_ons
        if was_active:
            self.controller.add_ons.remove(self.cap)
        try:
            return method(self, *args, **kwargs)
        finally:
            # restore it
            if was_active:
                self.controller.add_ons.append(self.cap)
    return wrapper

class ObjectPlacer:
    def __init__(
            self,
            output_path: str = "tdw_output",
            random_state: int = 42,
        ):
        self.random_state = np.random.RandomState(random_state)

        self.controller = Controller(launch_build=False)
        self.placed_objects = []  # List of dictionaries with object info and bboxes
        self.model_librarian = ModelLibrarian()


        # Set camera
        self.main_cam = ThirdPersonCamera(
            position={"x": -4.5, "y": 0.8, "z": -4.5},
            look_at={"x": 0, "y": 0.0, "z": 0},
            field_of_view=90,
            avatar_id="main_cam",
        )
        self.top_down_cam = ThirdPersonCamera(
            position={"x": 0, "y": 10, "z": 0},
            look_at={"x": 0, "y": 0, "z": 0},
            avatar_id="top_down_cam",
        )
        self.controller.add_ons.append(self.main_cam)
        self.controller.add_ons.append(self.top_down_cam)

        # Set image capture
        self.output_directory = Path(output_path)
        self.output_directory.mkdir(parents=True, exist_ok=True)
        print(f"Images will be saved to: {self.output_directory.resolve()}")
        self.cap = ImageCapture(
            path=self.output_directory,
            avatar_ids=["main_cam", "top_down_cam"],
            pass_masks=["_id", "_img"],
        )
        self.controller.add_ons.append(self.cap)

        
        
    @without_capture
    def _get_object_bounds_old(
            self,
            model_name: str,
            scale: dict = {"x": 1.0, "y": 1.0, "z": 1.0},
            rotation: dict | int = {"x": 0, "y": 0, "z": 0},
            room_size: tuple = (10, 10),
        ) -> tuple:
        """
        Get object bounding box by placing in extended room's temp area
        
        Args:
            model_name: Name of the object model in TDW library
            scale: {"x": float, "y": float, "z": float}
            rotation: {"x": float, "y": float, "z": float} or int: rotation in degrees
            room_size: tuple: (width, depth) of the room
            
        Returns:
            tuple: (width, depth) of the object bounding box

        NOTE the bounding box will also be rotated, the most left, right, front, ... points are also rotated.
        """
        if isinstance(rotation, int):
            rotation = {"x": 0, "y": rotation, "z": 0}

        # Place in temp area (center of the temporary (right) room)
        temp_obj_id = self.controller.get_unique_id()
        
        resp = self.controller.communicate([
            self.controller.get_add_object(
                model_name=model_name,
                position={'x': 0, 'y': 0, 'z': 0},
                rotation=rotation,
                object_id=temp_obj_id,
            ),
            {"$type": "scale_object", "id": temp_obj_id, "scale_factor": scale},
            {"$type": "send_bounds", "ids": [temp_obj_id], "frequency": "once"}
        ])
        
        # Get bounds
        bound = [Bounds(resp[i]) for i in range(len(resp) - 1) if OutputData.get_data_type_id(resp[i]) == 'boun'][0]
        print(f"Object {model_name} left: {bound.get_left(0)}, right: {bound.get_right(0)}, front: {bound.get_front(0)}, back: {bound.get_back(0)}")
        width = abs(bound.get_right(0)[0] - bound.get_left(0)[0])
        depth = abs(bound.get_front(0)[2] - bound.get_back(0)[2])
        bounds = (width, depth)
        print(f"Bounds of {model_name}: {bounds}")
        
        # Clean up temp object
        self.controller.communicate({"$type": "destroy_object", "id": temp_obj_id})

        return bounds
    
    def _get_object_bounds(
            self,
            model_name: str,
            scale: dict = {"x": 1.0, "y": 1.0, "z": 1.0},
            rotation: dict | int = {"x": 0, "y": 0, "z": 0},
        ) -> tuple:
        """
        Get object bounds
        """
        record = self.model_librarian.get_record(model_name)
        bounds = record.bounds # dict of left, right, front, back, bottom, top
        # Apply scale
        width = (bounds['right']['x'] - bounds['left']['x']) * scale['x']
        depth = (bounds['front']['z'] - bounds['back']['z']) * scale['z']
        
        # Apply rotation
        if isinstance(rotation, dict):
            rotation = rotation['y']
            
        # For y-axis rotation, width and depth are swapped based on angle
        angle = rotation % 360
        if angle in [0, 180]:
            pass
        elif angle in [90, 270]:
            width, depth = depth, width
        else:
            angle_rad = math.radians(angle)
            new_width = abs(width * math.cos(angle_rad)) + abs(depth * math.sin(angle_rad))
            new_depth = abs(width * math.sin(angle_rad)) + abs(depth * math.cos(angle_rad))
            width, depth = new_width, new_depth
            
        return (width, depth)
        
    
    def _check_overlap(self, x, z, width, depth, min_distance=0.5):
        """Check if position overlaps with existing objects (including min_distance)"""
        for obj_info in self.placed_objects:
            px, pz = obj_info['position']['x'], obj_info['position']['z']
            pw, pd = obj_info['bounds']
            if (abs(x - px) < (width + pw) / 2 + min_distance and 
                abs(z - pz) < (depth + pd) / 2 + min_distance):
                return True
        return False
    
    def _find_valid_position(
            self,
            width: float,
            depth: float,
            room_size: tuple,
            min_distance: float = 0.5,
            max_attempts: int = 100,
        ) -> tuple:
        """
        Find valid non-overlapping position
        
        Args:
            width: Width of the object
            depth: Depth of the object
            room_size: tuple: (width, depth) of the room
            min_distance: Minimum distance between objects
            max_attempts: Maximum number of attempts to find a valid position
            
        Returns:
            tuple: (x, z) of the valid position

        TODO: maintain a H x W grid, each cell holds a value indicating empty space around it
        """
        for _ in range(max_attempts):
            x = int(self.random_state.randint(int(-room_size[0] / 2 + width / 2), 
                              int(room_size[0] / 2 - width / 2)))
            z = int(self.random_state.randint(int(-room_size[1] / 2 + depth / 2), 
                              int(room_size[1] / 2 - depth / 2)))
            
            if not self._check_overlap(x, z, width, depth, min_distance):
                return x, z
        return None, None
    
    def place_objects(
            self, 
            room_size: tuple, 
            object_pool: list, 
            num_objects: int, 
            min_distance=0.5, 
            screen_size: tuple = (2048, 2048),
        ) -> None:
        """Main function to place objects in room"""
        # Create extended room with extra padding space (wall)
        PAD = 1
        self.controller.communicate([
            TDWUtils.create_empty_room(room_size[0] + PAD, room_size[1] + PAD),
            {"$type": "set_screen_size", "width": screen_size[0], "height": screen_size[1]},
        ])

        
        placed_count = 0
        attempts = 0
        max_total_attempts = num_objects * 10
        
        while placed_count < num_objects and attempts < max_total_attempts:
            attempts += 1
            
            # Random object and properties
            model = self.random_state.choice(object_pool)
            
            scale = {"x": 1.0, "y": 1.0, "z": 1.0} # TODO: use scale to constrain the scale of the object
            rotation = {"x": 0, "y": int(self.random_state.choice([0, 90, 180, 270])), "z": 0}
            
            # Get object bounds
            bounds = self._get_object_bounds(model, scale, rotation)
            print(f"Bounds of {model}: {bounds}")
            if not bounds:
                continue
                
            width, depth = bounds
            
            # Find valid position
            x, z = self._find_valid_position(width, depth, room_size, min_distance, max_attempts=50)
            if x is None:
                continue
            
            # Place object in main room
            obj_id = self.controller.get_unique_id()
            self.controller.communicate([
                self.controller.get_add_object(
                    model_name=model,
                    position={'x': x, 'y': 0, 'z': z},
                    rotation=rotation,
                    object_id=obj_id,
                ),
                {"$type": "scale_object", "id": obj_id, "scale_factor": scale},
            ])
            
            # Store detailed placement info
            obj_info = {
                'id': obj_id,
                'name': f"{model}_{obj_id}",
                'model': model,
                'position': {'x': x, 'y': 0, 'z': z},
                'rotation': rotation,
                'scale': scale,
                'bounds': (width, depth)
            }
            self.placed_objects.append(obj_info)
            placed_count += 1
            print(f"Placed {model} at ({x}, {z}) with rotation {rotation}°")
        
        print(f"Successfully placed {placed_count}/{num_objects} objects")
        print(f"Placed objects: {self.placed_objects}")
        return self.controller
    
    def move_camera(self,
            position: dict = None, 
            rotation: dict = None,
            look_at: dict | int = None,
        ) -> None:
        """
        Move the main camera to a new position and/or rotation.
        
        Args:
            position: Dictionary with x, y, z coordinates for camera position
            rotation: Dictionary with x, y, z rotation angles in degrees
            look_at: Dictionary of position or object_id to look at
        """
        commands = []
        
        if position is not None:
            commands.append({
                "$type": "teleport_avatar_to",
                "avatar_id": "main_camera", 
                "position": position
            })
            
        if rotation is not None:
            commands.append({
                "$type": "rotate_avatar_to",
                "avatar_id": "main_camera",
                "rotation": rotation
            })
            
        if look_at is not None:
            self.main_cam.look_at(look_at)
            
        if commands:
            self.controller.communicate(commands)
    
    def cleanup(self):
        """Clean up and terminate the simulation."""
        self.controller.communicate({"$type": "terminate"})
        print("Simulation terminated.")

In [4]:
# Usage example
if __name__ == "__main__":
    # Example object pool (replace with actual TDW model names)
    object_pool = ['blue_club_chair', 'blue_side_chair', 'brown_leather_dining_chair', 'brown_leather_side_chair', 'chair_annabelle', 'chair_billiani_doll', 'chair_eames_plastic_armchair', 'chair_thonet_marshall', 'chair_willisau_riale', 'dark_red_club_chair', 'emeco_navy_chair', 'green_side_chair', 'lapalma_stil_chair', 'ligne_roset_armchair', 'linbrazil_diz_armchair', 'linen_dining_chair', 'naughtone_pinch_stool_chair', 'red_side_chair', 'tan_lounger_chair', 'tan_side_chair', 'vitra_meda_chair', 'white_club_chair', 'white_lounger_chair', 'wood_chair', 'yellow_side_chair']
    
    placer = ObjectPlacer("my_output")
    controller = placer.place_objects(
        room_size=(5, 5),
        object_pool=object_pool, 
        num_objects=5, 
        min_distance=1.0
    )
    
    placer.cleanup()


Your installed tdw Python module is up to date with PyPi.
You need to launch your own build.


Images will be saved to: /home/pingyue/Work/spatial/simulator/TDW/spatial/debug/my_output
Bounds of chair_eames_plastic_armchair: (0.6054678, 0.6331428)
Placed chair_eames_plastic_armchair at (-2, 0) with rotation {'x': 0, 'y': 270, 'z': 0}°
Bounds of emeco_navy_chair: (0.5086646, 0.4200468)
Placed emeco_navy_chair at (-2, -2) with rotation {'x': 0, 'y': 270, 'z': 0}°
Bounds of chair_eames_plastic_armchair: (0.6054678, 0.6331428)
Placed chair_eames_plastic_armchair at (0, 0) with rotation {'x': 0, 'y': 90, 'z': 0}°
Bounds of emeco_navy_chair: (0.4200468, 0.5086646)
Placed emeco_navy_chair at (1, -2) with rotation {'x': 0, 'y': 180, 'z': 0}°
Bounds of brown_leather_side_chair: (0.6301832, 0.6674332000000001)
Bounds of chair_thonet_marshall: (0.3967712, 0.48217699999999997)
Bounds of tan_lounger_chair: (1.5227728999999999, 1.0091138)
Bounds of blue_side_chair: (0.6296079, 0.620476)
Bounds of chair_willisau_riale: (0.5401487, 0.4736602)
Bounds of blue_side_chair: (0.6296079, 0.620476)
Bou